# Computational Phase Retrieval with Tensor Methods

## Device Information

In [1]:
!nvidia-smi

Tue Jul 13 10:47:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.80       Driver Version: 460.80       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 165...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   48C    P8     8W /  N/A |    301MiB /  3911MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Import Required Libraries

In [109]:
import tensorflow as tf
print(f"Tensorflow version: {tf.__version__}")

import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        print("Num GPUs Available: ", len(gpus))
        for gpu in gpus:
            # Allow memory growth for the GPU.
            # Reference: https://www.tensorflow.org/guide/gpu
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized.
        print(e)

from matplotlib import pyplot as plt
plt.style.use('dark_background')
import numpy as np
import tensorly as tl
import cv2
import time
import os
from PIL import Image

import matplotlib.cm as cm
import seaborn as sns

from matplotlib import rc, rcParams
rcParams['font.family'] = 'Cubano'
rc('text', usetex=False)

import warnings

Tensorflow version: 2.5.0


## GPU Benchmark

Reference: https://www.tensorflow.org/guide/gpu

In [5]:
tf.debugging.set_log_device_placement(True)

n = 5000
num_iters = 10

'''
Test with TensorFlow GPU.
'''
start_tf = time.time()

for i in range(num_iters):
    # Tensors are defaultly placed on the GPU (CPU would be considerably slower due
    # to the incurred communication cost).
    # with tf.device('/CPU:0'):
    a = tf.ones((n, n))
    b = tf.ones((n, n))

    # Run on the GPU
    c = tf.matmul(a, b)

print(f'Elapsed time with TensorFlow GPU: {time.time() - start_tf}')

'''
Test with Numpy.
'''
start_np = time.time()

# for i in range(num_iters):
#     a = np.ones((n, n))
#     b = np.ones((n, n))

#     c = np.dot(a, b)

print(f'Elapsed time with Numpy: {time.time() - start_np}') # CAN BE SLOW


Elapsed time with TensorFlow GPU: 0.5627357959747314
Elapsed time with Numpy: 15.692136526107788


## Low Rank Phase Retrieval

References:

\[1\] Namrata Vaswani, Seyedehsara Nayer, Yonina C. Eldar. *Low Rank Phase Retrieval*. https://rutgers.box.com/s/dntl0sh157p62rgi1zerdaxrqthugr32

\[2\] Namrata Vaswani. *Nonconvex Structured Phase Retrieval*. https://rutgers.box.com/s/x02w8frd1ep01cxdjlnojufa9npvstsz.

\[3\] Tamara G. Kolda, Brett W. Bader. *Tensor Decompositions and Applications*. https://rutgers.box.com/s/aq9psx3mgwhms6rrzlhn94h56c3oshox. 




### Define Data Directories

In [110]:
INPUT_DIR = './videos/' # directory of the test videos
OUTPUT_DIR = './output/' # output directory
FRAMES_DIR = './ouput/frames/' # output directory of the extracted video frames 

### Load the Test Video

In [111]:
# Read the video.
video_path = INPUT_DIR + os.listdir(INPUT_DIR)[0] # define video path
cap = cv2.VideoCapture(video_path) # read the video from path
video_name = os.listdir(INPUT_DIR)[0].split('.')[0] # get the name of the video

# Creat the folder to store the extracted frames of the video.
try:
    if not os.path.exists(FRAMES_DIR + video_name):
        os.makedirs(FRAMES_DIR + video_name)
    else:
        print('Directory already exists!')
except OSError:
    print('OS ERROR')

k = 0 # frame number, k = 0, 1, 2, ..., q - 1
Xlist = []
while (True):
    # Capture the video frame-by-frame.
    # Code adopted: https://docs.opencv.org/3.4/dd/d43
    # # tutorial_py_video_display.html
    ret, frame = cap.read()

    # If the frame is read correctly the return boolean (ret) is true.
    if not ret:
        print("Cannot receive frame (probably end of stream). Exiting...")
        break
    else:
        # Convert the frame to grayscale.
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        name = FRAMES_DIR + video_name + '/frame-' + str(k) + '.jpg'
        print('DEBUG: Captured...' + name)
        cv2.imwrite(name, gray_frame)
        Xlist.append(gray_frame)

        k += 1

# Release the capture when finished.
cap.release()
cv2.destroyAllWindows()

Directory already exists!
DEBUG: Captured..../ouput/frames/sara/frame-0.jpg
DEBUG: Captured..../ouput/frames/sara/frame-1.jpg
DEBUG: Captured..../ouput/frames/sara/frame-2.jpg
DEBUG: Captured..../ouput/frames/sara/frame-3.jpg
DEBUG: Captured..../ouput/frames/sara/frame-4.jpg
DEBUG: Captured..../ouput/frames/sara/frame-5.jpg
DEBUG: Captured..../ouput/frames/sara/frame-6.jpg
DEBUG: Captured..../ouput/frames/sara/frame-7.jpg
DEBUG: Captured..../ouput/frames/sara/frame-8.jpg
DEBUG: Captured..../ouput/frames/sara/frame-9.jpg
DEBUG: Captured..../ouput/frames/sara/frame-10.jpg
DEBUG: Captured..../ouput/frames/sara/frame-11.jpg
DEBUG: Captured..../ouput/frames/sara/frame-12.jpg
DEBUG: Captured..../ouput/frames/sara/frame-13.jpg
DEBUG: Captured..../ouput/frames/sara/frame-14.jpg
DEBUG: Captured..../ouput/frames/sara/frame-15.jpg
DEBUG: Captured..../ouput/frames/sara/frame-16.jpg
DEBUG: Captured..../ouput/frames/sara/frame-17.jpg
DEBUG: Captured..../ouput/frames/sara/frame-18.jpg
DEBUG: Captured

### Create the true signal tensor.

Tensors are multi-dimensional arrays with a uniform type (`dtype`). All tensors are immutable like Python numbers and strings: you can never update the contents of a tensor, only create a new one.

**Note**: In libraries like tensorflow, the rank of the tensor actually denotes the order of the tensor in our convention. We call the `rank` of a tensor in a similar manner as the rank of a matrix.

The gray-scaled signal is modeled as a three-ordered tensor $\boldsymbol{\mathcal{X}} \in \mathbb{R}^{I_1 \times I_2 \times q}$, where $I_1 \times I_2$ correspond to the pixel coordinates within each frame and $q$ is the total number of frames captured.

**Signal Dimension**

In [112]:
Xast = tf.constant(Xlist, tf.double)
q, I1, I2 = Xast.shape
print(f'The dimension of the true signal tensor: I1 x I2 x q: {I1} x {I2} x {q}')

The dimension of the true signal tensor: I1 x I2 x q: 480 x 640 x 105


### Generate Phaseless Measurements

In [113]:
A = [] # measurement tensor in list format
m = 10 # number of measurements

np.random.seed(5) # set random seed

# Generate i.i.d. measurement tensors.
for j in range(m):
    Aj = []
    for k in range(q):
        Ajk = np.random.randn(I1, I2) # i.i.d. normal measurement
        Aj.append(Ajk)
    A.append(Aj)

In [114]:
# Generate phaseless measurements.
Y = np.zeros((m, q)) # matrix of the phaseless measurements

for j in range(m):
    for k in range(q):
        Y[j, k] = tf.tensordot(A[j][k], Xast[k], axes=([0, 1], [0, 1]))**2

In [115]:
print('DEBUG')
print(f'\nDimension of the phaseless measurement matrix: m x q: {Y.shape[0]} x {Y.shape[1]}\n')
print('Phaseless Measurements:\n\n', Y)

DEBUG

Dimension of the phaseless measurement matrix: m x q: 10 x 105

Phaseless Measurements:

 [[1.27160654e+08 1.09215703e+10 1.05717725e+10 ... 1.08445766e+10
  1.71644657e+08 6.02085249e+07]
 [4.58334424e+09 4.13127136e+08 1.12115112e+08 ... 1.56249743e+09
  3.07167905e+06 2.64488749e+09]
 [1.61810815e+09 7.29115357e+08 8.63181443e+09 ... 1.63338103e+09
  4.72147160e+08 4.21422069e+06]
 ...
 [9.36276208e+09 2.34494775e+07 5.31417782e+09 ... 8.60596714e+07
  2.24366966e+10 9.15723194e+09]
 [3.98034489e+09 2.67960132e+08 1.04974115e+10 ... 9.55799810e+08
  1.46223399e+09 2.42790383e+09]
 [6.47884266e+06 3.99879918e+09 3.52638316e+09 ... 5.97114690e+08
  1.24974284e+09 1.25169085e+09]]


In [116]:
def initialize(I1, I2, q, R):
    U1 = np.random.randn(I1, rank)
    U2 = np.random.randn(I2, rank)
    U = [U1,U2]
    B = np.random.randn(rank, q)

    return U, B

In [117]:
def kruskal(U, B, R, Lambda=None, type='CP'):
    """Construct Tensor from Kruskal Formulation.

        Args:
            U: list consisting of two factor matrices U1 (I1 x R)
                and U2 (I2 x R) for the three-way case.
            B: the B (q x R) factor matrix.
            R: assumped rank (a scalar) of the low-rank tensor.
            Lambda: normalization factors (length R).
        
        Returns:
            Xhat: signal estimate (I1 x I2 x q).
    """
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    if type == 'CP':
        U1, U2 = U[0], U[1]
        I1, I2, q = U1.shape[0], U2.shape[0], B.shape[0]
        Xhat = tf.zeros([I1, I2, q])
        if Lambda is None:
            Lambda = tf.ones([R,])
        for r in range(R):
            U1U2 = tf.tensordot(U1[:, r], U2[:, r], axes=0)
            Xhat += Lambda[r] * tf.tensordot(U1U2, B[:, r], axes=0)
        
        return Xhat
    else:
        return None

In [130]:
from tensorly.decomposition import parafac

X = tl.tensor(np.arange(24).reshape((3, 4, 2)), dtype=tl.float32)
weights, factors = parafac(X, rank=3)

X_test = kruskal([factors[0], factors[1]], factors[2], 3, weights)

err = X - X_test

print(f'DEBUG | Mean squared error for reconstruction: {tf.math.reduce_sum(tf.multiply(err, err)).numpy()}')



DEBUG | Mean squared error for reconstruction: 2.1168126806969667e-07


In [51]:
def descent(U, B, A, Y, R, max_iter):
    Xhat = kruskal(U, B, R)
    for _ in range(max_iter):
        for k in range(q):
            Ax = []
            for j in range(m):
                Ax.append(tf.tensordot(A[j][k], X_ast[k], axes=([0, 1], [0, 1])))
            Ck = tf.linalg.diag(tf.angle(Ax))
            U[0] = least_squares_solver()
    
    return U, Xhat

In [ ]:
def tensor_lrpr(A, Y, rank=10, max_iter=5):
    U, B = initialize(I1, I2, q, R)
    
    return descent(U, B, A, Y, R, max_iter)

In [ ]:
class TensorLRPR

In [ ]:
class TensorUtils